## Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# tensorflow
import tensorflow as tf
print('Tested with TensorFlow 1.2.0')
print ('Your TensorFlow version:', tf.__version__) # tested with v1.2
from tensorflow.contrib.learn.python.learn.estimators import constants
from tensorflow.contrib.learn.python.learn.estimators.dynamic_rnn_estimator import PredictionType

# Estimators
from tensorflow.contrib import learn

# Model builder
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from tensorflow.contrib.learn.python.learn.estimators import rnn_common


# Input function
from tensorflow.python.estimator.inputs import numpy_io

# data and data preprocessing
from tensorflow.contrib.keras.python.keras.datasets import imdb 
from tensorflow.contrib.keras.python.keras.preprocessing import sequence

# Run an experiment
from tensorflow.contrib.learn.python.learn import learn_runner

# numpy
import numpy as np

# Enable TensorFlow logs
tf.logging.set_verbosity(tf.logging.INFO)

Tested with TensorFlow 1.2.0
Your TensorFlow version: 1.2.1


## Helpers

In [2]:
# map word to index
word_to_index = imdb.get_word_index()
# map index to word
index_to_word = {}
num_words = 0
for k in word_to_index: 
    index_to_word[word_to_index[k]] = k
    num_words += 1

# turn a sequence into a sentence
def get_sentence(seq):
    sentence = ''
    for v in seq:
        if v != 0: # 0 means it was just added to the sentence so it could have maxlen words
            sentence += index_to_word[int(v)] + ' '
    return sentence

# turn a sentence into a sequence
def gen_sequence(sentence):
    seq = []
    for word in sentence:
        seq.append(word_to_index[word])
    return np.asarray(seq, dtype=np.float32)

print('there are', num_words, 'words in the files')
#for i in range(1, 100):
#    print(i, index_to_word[i])
#print(index_to_word[49], index_to_word[212], index_to_word[242], index_to_word[136])

there are 88584 words in the files


## Visualizing data

In [3]:
'''
# ------------------- negative
print('-' * 30)
print('Example of a negative review')
print('-' * 30)

x = open('data/train/neg/0_3.txt')
r = x.readline()
print(r)

# ------------------ positive
print()
print('-' * 30)
print('Example of a positive review')
print('-' * 30)

x = open('data/train/pos/0_9.txt')
r = x.readline()
print(r)
'''

"\n# ------------------- negative\nprint('-' * 30)\nprint('Example of a negative review')\nprint('-' * 30)\n\nx = open('data/train/neg/0_3.txt')\nr = x.readline()\nprint(r)\n\n# ------------------ positive\nprint()\nprint('-' * 30)\nprint('Example of a positive review')\nprint('-' * 30)\n\nx = open('data/train/pos/0_9.txt')\nr = x.readline()\nprint(r)\n"

In [4]:
print('Loading data')
NUM_WORDS = 1000
MAX_LENGTH = 250

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)


Loading data


In [5]:
x_train.size

25000

In [6]:
# parameters
LEARNING_RATE = 0.001
BATCH_SIZE = 10

def get_input_fn(x_in, y_in, shuffle=True, epochs=1):
    def length_bin(length, length_step=10, maxlen=MAX_LENGTH):
        # This function sets the sequence-length bin.
        # The returned value is the length of the longest sequence allowed in the bin.
        bin_id = (length // length_step + 1)*length_step
        return tf.cast(tf.minimum(bin_id, maxlen),tf.int64)
    
    def make_batch(key, ds):
        # Eliminate the extra padding
        ds = ds.map(lambda x,x_len,y: (x[:tf.cast(key,tf.int32)],x_len, y))
        
        # convert the entire contents of the bin to a batch
        ds = ds.batch(BATCH_SIZE)
        return ds
        
    
    def input_fn():
        ys = tf.contrib.data.Dataset.from_tensor_slices(y_in)
        
        x_len = np.minimum(np.array([len(seq) for seq in x_in]), MAX_LENGTH).astype('int32')
        
        # DynalicRNNEstimator uses `rnn_common.select_last_activations`
        #     https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/contrib/learn/python/learn/estimators/dynamic_rnn_estimator.py
        # So we want to add padding at the end of the sequence, the default is the start of the sequence:
        #     https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/keras/preprocessing/sequence/pad_sequences
        xs = sequence.pad_sequences(x_in, maxlen=MAX_LENGTH, padding='post')
        
        ds = tf.contrib.data.Dataset.from_tensor_slices((xs, x_len, y_in))
        
        ds = ds.repeat(epochs)
        
        if shuffle:
            ds = ds.shuffle(10000)

        ds = ds.group_by_window(
            key_func=lambda x, x_len, y: length_bin(x_len), 
            reduce_func=make_batch,
            window_size=BATCH_SIZE)
        
        x,x_len,y = ds.make_one_shot_iterator().get_next()

        return {'x':x, rnn_common.RNNKeys.SEQUENCE_LENGTH_KEY:x_len},y
        
    return input_fn

In [7]:
train_input_fn = get_input_fn(x_train, y_train, BATCH_SIZE, epochs=100)
test_input_fn = get_input_fn(x_test, y_test, BATCH_SIZE, epochs=1)

In [8]:
x,y = train_input_fn()

with tf.train.MonitoredSession() as sess:
    for n in range(30):
        x_,y_ = sess.run([x,y])

In [9]:
x

{'sequence_length': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>,
 'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?) dtype=int32>}

In [10]:
x_

{'sequence_length': array([117, 118, 110, 111, 111, 115, 116, 118, 116, 112], dtype=int32),
 'x': array([[  1, 207, 110, ...,   0,   0,   0],
        [  1,  14,  20, ...,   2,   0,   0],
        [  1,  14,  22, ...,   0,   0,   0],
        ..., 
        [  1,  14,  20, ...,  20,   0,   0],
        [  1,  45,  55, ...,   0,   0,   0],
        [  1,  91, 102, ...,   0,   0,   0]], dtype=int32)}

In [11]:
x_['x'].shape

(10, 120)

In [12]:
x_['x']

array([[  1, 207, 110, ...,   0,   0,   0],
       [  1,  14,  20, ...,   2,   0,   0],
       [  1,  14,  22, ...,   0,   0,   0],
       ..., 
       [  1,  14,  20, ...,  20,   0,   0],
       [  1,  45,  55, ...,   0,   0,   0],
       [  1,  91, 102, ...,   0,   0,   0]], dtype=int32)

In [13]:
y

<tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=int64>

In [14]:
xc = tf.contrib.layers.sparse_column_with_integerized_feature('x',NUM_WORDS)
xc = tf.contrib.layers.embedding_column(xc,30)

In [ ]:

estimator = tf.contrib.learn.DynamicRnnEstimator(
     model_dir='tensorboard/RNN_dropout_2',
     problem_type = constants.ProblemType.CLASSIFICATION,
     prediction_type = PredictionType.SINGLE_VALUE,
     sequence_feature_columns = [xc],
     context_feature_columns = None,
     num_units = [256, 128],
     cell_type = 'lstm', 
     optimizer = 'Adam',
     learning_rate = LEARNING_RATE,
     num_classes = 2,
     dropout_keep_probabilities=[0.9,0.9,0.9])

# create experiment
def generate_experiment_fn():
  
  """
  Create an experiment function given hyperparameters.
  Returns:
    A function (output_dir) -> Experiment where output_dir is a string
    representing the location of summaries, checkpoints, and exports.
    this function is used by learn_runner to create an Experiment which
    executes model code provided in the form of an Estimator and
    input functions.
    All listed arguments in the outer function are used to create an
    Estimator, and input functions (training, evaluation, serving).
    Unlisted args are passed through to Experiment.
  """

  def _experiment_fn(output_dir):

    train_input = train_input_fn
    test_input = test_input_fn
    
    return tf.contrib.learn.Experiment(
        estimator,
        train_input_fn=train_input,
        eval_input_fn=test_input,
    )
  return _experiment_fn

# run experiment 
learn_runner.run(generate_experiment_fn(), output_dir='/tmp/sentiment_analysis')

Instructions for updating:
This file will be removed after the deprecation date.Please switch to third_party/tensorflow/contrib/learn/python/learn/estimators/head.py
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc98b632fd0>, '_model_dir': 'tensorboard/RNN_dropout_2', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 int

INFO:tensorflow:loss = 0.640791, step = 2301 (20.912 sec)
INFO:tensorflow:global_step/sec: 4.73134
INFO:tensorflow:loss = 0.634194, step = 2401 (21.136 sec)
INFO:tensorflow:global_step/sec: 4.90182
INFO:tensorflow:loss = 0.39333, step = 2501 (20.400 sec)
INFO:tensorflow:global_step/sec: 4.89397
INFO:tensorflow:loss = 0.279892, step = 2601 (20.433 sec)
INFO:tensorflow:global_step/sec: 4.79752
INFO:tensorflow:loss = 0.491476, step = 2701 (20.844 sec)
INFO:tensorflow:global_step/sec: 4.96981
INFO:tensorflow:loss = 0.715427, step = 2801 (20.121 sec)
INFO:tensorflow:Saving checkpoints for 2859 into tensorboard/RNN_dropout_2/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-07-05-10:31:44
INFO:tensorflow:Restoring parameters from tensorboard/RNN_dropout_2/model.ckpt-2859
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:ten

INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/1

INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [55/100]
INFO:tensorflow:Evaluation [56/100]
INFO:tensorflow:Evaluation [57/100]
INFO:tensorflow:Evaluation [58/100]
INFO:tensorflow:Evaluation [59/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [61/100]
INFO:tensorflow:Evaluation [62/100]
INFO:tensorflow:Evaluation [63/100]
INFO:tensorflow:Evaluation [64/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [71/100]
INFO:tensorflow:Evaluation [72/100]
INFO:tensorflow:Evaluation [73/100]
INFO:tensorflow:Evaluation [74/100]
INFO:tensorflow:Evaluation [75/100]
INFO:tensorflow:Evaluation [76/100]
INFO:tensorflow:Evaluation [77/100]
INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [

...

INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [55/100]
INFO:tensorflow:Evaluation [56/100]
INFO:tensorflow:Evaluation [57/100]
INFO:tensorflow:Evaluation [58/100]
INFO:tensorflow:Evaluation [59/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [61/100]
INFO:tensorflow:Evaluation [62/100]
INFO:tensorflow:Evaluation [63/100]
INFO:tensorflow:Evaluation [64/100]
INFO:tensorflow:Evaluation [65/100]
INFO:tensorflow:Evaluation [66/100]
INFO:tensorflow:Evaluation [67/100]
INFO:tensorflow:Evaluation [68/100]
INFO:tensorflow:Evaluation [69/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [71/100]
INFO:tensorflow:Evaluation [72/100]
INFO:tensorflow:Evaluation [73/100]
INFO:tensorflow:Evaluation [74/100]
INFO:tensorflow:Evaluation [75/100]
INFO:tensorflow:Evaluation [76/100]
INFO:tensorflow:Evaluation [77/100]
INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-07-05-12:21:59
INFO:tensorflow:Saving dict for global step 34062: accuracy = 0.887, global_step = 34062, loss = 0.286793
INFO:tensorflow:Validation (step 34062): loss = 0.286793, global_step = 34062, accuracy = 0.887
INFO:tensorflow:global_step/sec: 2.74727
INFO:tensorflow:loss = 0.081937, step = 34101 (36

INFO:tensorflow:global_step/sec: 4.24513
INFO:tensorflow:loss = 0.204351, step = 37101 (23.556 sec)
INFO:tensorflow:global_step/sec: 3.74752
INFO:tensorflow:loss = 0.0707189, step = 37201 (26.684 sec)
INFO:tensorflow:global_step/sec: 4.27873
INFO:tensorflow:loss = 0.203173, step = 37301 (23.371 sec)
INFO:tensorflow:global_step/sec: 4.04459
INFO:tensorflow:loss = 0.193706, step = 37401 (24.724 sec)
INFO:tensorflow:global_step/sec: 3.75429
INFO:tensorflow:loss = 0.07781, step = 37501 (26.636 sec)
INFO:tensorflow:global_step/sec: 4.42738
INFO:tensorflow:loss = 0.102203, step = 37601 (22.587 sec)
INFO:tensorflow:global_step/sec: 4.21916
INFO:tensorflow:loss = 0.06468, step = 37701 (23.702 sec)
INFO:tensorflow:global_step/sec: 4.09393
INFO:tensorflow:loss = 0.0520879, step = 37801 (24.426 sec)
INFO:tensorflow:global_step/sec: 3.86305
INFO:tensorflow:loss = 0.31497, step = 37901 (25.886 sec)
INFO:tensorflow:global_step/sec: 4.26118
INFO:tensorflow:loss = 0.196783, step = 38001 (23.468 sec)
I

INFO:tensorflow:loss = 0.0351918, step = 41001 (20.429 sec)
INFO:tensorflow:global_step/sec: 4.85861
INFO:tensorflow:loss = 0.110542, step = 41101 (20.582 sec)
INFO:tensorflow:global_step/sec: 4.72757
INFO:tensorflow:loss = 0.162506, step = 41201 (21.152 sec)
INFO:tensorflow:global_step/sec: 4.90204
INFO:tensorflow:loss = 0.126622, step = 41301 (20.400 sec)
INFO:tensorflow:global_step/sec: 4.39313
INFO:tensorflow:loss = 0.109255, step = 41401 (22.763 sec)
INFO:tensorflow:global_step/sec: 4.30744
INFO:tensorflow:loss = 0.161081, step = 41501 (23.216 sec)
INFO:tensorflow:global_step/sec: 4.35636
INFO:tensorflow:loss = 0.0378314, step = 41601 (22.955 sec)
INFO:tensorflow:global_step/sec: 4.38115
INFO:tensorflow:loss = 0.106896, step = 41701 (22.825 sec)
INFO:tensorflow:Saving checkpoints for 41752 into tensorboard/RNN_dropout_2/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-07-05-12:51:49
INFO:tensorflow:Restoring parameters from tensorboard/RNN_dropout_2/model.ckpt-41752
INFO:te

In [ ]:
estimator = tf.contrib.learn.DynamicRnnEstimator(
     model_dir='tensorboard/RNN',
     problem_type = constants.ProblemType.CLASSIFICATION,
     prediction_type = PredictionType.SINGLE_VALUE,
     sequence_feature_columns = [xc],
     context_feature_columns = None,
     num_units = [256, 128],
     cell_type = 'lstm', 
     optimizer = 'Adam',
     learning_rate = LEARNING_RATE,
     num_classes = 2)

In [ ]:
# generate predictions
preds = list(estimator.predict(input_fn=test_input_fn))

# number of outputs we want to see the prediction
NUM_EVAL = 10
def check_prediction(x, y, p, index):
    print('prediction:', p[index]['classes'])
    print('target:', y[index])
    print('sentence:', get_sentence(x[index]))

for i in range(NUM_EVAL):
    index = np.random.randint(limit)
    print('test:', index)
    print('-' * 30)
    print(np.asarray(x_test[index], dtype=np.int32))
    check_prediction(x_test, y_test, preds, index)
    print()